In [1]:
import pandas as pd
import json
from tqdm import tnrange, tqdm_notebook

In [2]:
f = open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/for_coding2.json', 'r', encoding='UTF8')    # type3를 통해 추출된 json파일 읽어들이기
original_exp = json.load(f)
key_list = sorted(list(original_exp.keys()), key=len)    # 어려운 단어 리스트. 글자수가 더 많은 키워드를 먼저 ^^로 태깅하면 문제가 생김. 그래서 오름차순 정렬함

In [3]:
# 단어 하나가 문장에 여러번 등장했을때, 모든 위치를 리스트로 뽑아내는 함수.
# list씌워야함

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches
        
# ex) list(find_all('안녕안녕하세요', '안녕')) --> [0,2]

In [4]:
# isdiff 두 문장을 갖고와서 같으면 0, 다르면 1을 출력하는 함수.
def isdiff(sent_a, sent_b):
    if sent_a == sent_b:
        return 0
    else:
        return 1

In [5]:
# 숫자 리스트 받아와서 이에 맞게 다른 리스트의 순서 바꿔주는 함수.
def sort_word_wordOcc(word, wordOccurence):    # 순서 바꿀 단어 리스트, 순서 알려주는 리스트
#     try:
    if len(word) == len(wordOccurence):
        result = {}
        for i in range(len(word)):
            result.update({wordOccurence[i]['start']:word[i]})    # result = {wordOccurence:['start']:호환}

        sorted_result = sorted(result.items())     # ex) [('1','원심'), (17:'원심'), (127:'호환'), (172:'호환'), (320:'원심')]
        result_word = []
        result_wordOcc = []

        for i in range(0, len(word)):
            result_word.append(sorted_result[i][1])
            result_wordOcc.append({'start':sorted_result[i][0], 'end':sorted_result[i][0]+len(sorted_result[i][1])+1})
        
    else:
        raise Exception('word와 word_Occurence 간 크기가 다릅니다.')
        
#     except Exception as e:
#         print("예외가 발생했습니다.", e)
        
    return result_word, result_wordOcc

In [6]:
# 중복제거
def remove_overlap(overlapped_list):
    result = []
    for i in overlapped_list:
        if i not in result:
            result.append(i)
            
    return result

In [7]:
result = {}
tag = '^'

for i in tnrange(len(key_list)):
    result.update({key_list[i]:[]})
    
    for j in range(len(original_exp[key_list[i]])):
        sentence = original_exp[key_list[i]][j]["sentence"]
        paragraph = original_exp[key_list[i]][j]["paragraph"]
        keyword = original_exp[key_list[i]][j]["keyword"]
        keywordOccurence = []
        
        for k in range(len(paragraph)):
            sentence = sentence.replace(paragraph[k], tag + keyword[k] + tag)
            
        for q in range(len(paragraph)):
            start = list(find_all(sentence, keyword[q]))
            end_only_keyword = start[l]+len(keyword[k])
            
            for l in range(len(start)):
                keywordOccurence.append({'start':start[l]-1, 'end':end})
            
        result[key_list[i]].append({'sentence':sentence, 'paragraph': paragraph, 'keyword':keyword, 'keywordOccurence':keywordOccurence})
        
#         # 키데이터 중 문장에 있는 키워드 찾아내기
#         for k in range(len(key_list)):    # 모든 키워드에 대해 시행
#             if sentence[0:len(key_list[k])] == key_list[k]:    # 문장 첫 단어로 키워드가 있을 때..
#                 tagged_keyword = tag + data1[3] + tag
#                 sentence = sentence.replace(key_list[k], tagged_keyword)
#                 count = len(list(find_all(sentence, key_list[k])))
#                 result[key_list[i]][j].update({'sentence':sentence})
#                 for l in range(count):
#                     result[key_list[i]][j]['keyword'].append(key_list[k])
                
                
#             elif sentence.find(' '+key_list[k]) != -1:    # 문장 첫 단어로 키워드가 있을 때..
#                 tagged_keyword = tag + key_list[k] + tag
#                 sentence = sentence.replace(key_list[k], tagged_keyword)
#                 count = len(list(find_all(sentence, key_list[k])))
#                 result[key_list[i]][j].update({'sentence':sentence})
#                 for l in range(count):
#                     result[key_list[i]][j]['keyword'].append(key_list[k])
                
#             else:
#                 pass

<ipython-input-7-8c697acb1443>:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# word와 wordOccurence 직관적으로 정렬.
for i in tnrange(len(key_list)):
    for j in range(len(original_exp[key_list[i]])):
        paragraph = result[key_list[i]][j]['paragraph']
        keyword = result[key_list[i]][j]['keyword']
        tagged_keyword_pos = result[key_list[i]][j]['keywordOccurence']
        result_word, result_wordOcc = sort_word_wordOcc(paragraph, tagged_keyword_pos)
        result[key_list[i]][j]['paragraph'] = result_word
        result_word, result_wordOcc = sort_word_wordOcc(keyword, tagged_keyword_pos)
        result[key_list[i]][j]['keyword'] = result_word
        result[key_list[i]][j]['keywordOccurence'] = result_wordOcc

<ipython-input-9-cb0679df4dae>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# json 파일로 저장
with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_B_input.json', 'w', encoding='UTF8') as json_file:
# with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_A_input_example.json', 'w', encoding='UTF8') as json_file:
    json.dump(result, json_file, indent=4, ensure_ascii=False,)